In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf

### Loading the data set

In [15]:
data = pd.read_csv('train.csv').as_matrix()

data.shape

C:\Users\prince.yadav\AppData\Local\Continuum\miniconda3\envs\semantic\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


(42000, 785)

### Data Suffle

In [11]:
y, X = data[:,0], data[:,1:]
y = y.reshape([-1,1])
indices = list(range(len(X)))
np.random.shuffle(indices)

X = X[indices]
y = y[indices]

print("Suffle done")

Suffle done


### Splitting into train and test data

In [21]:
n_train = int(0.7 * len(y))

y_train, X_train, y_test, X_test = y[: n_train], X[: n_train, :], y[n_train:], X[n_train:, :]

X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)

print("Train data length: ", len(X_train))

Train data length:  29399


### Data Normalization

In [22]:
## Normalization (x - mean)/mean

X_train -= 127.0
X_train /= 127.0

X_test  -= 127.0
X_test /= 127.0

### Building a tensorflow graph

In [29]:
## Creating a batches of images

def get_next_batch(batch_size, X, y):
    for i in range(0, len(y), batch_size):
        start = i
        end = min(len(y), i+batch_size)
        
        ## in case of yield for loop will be stuck at place from where function terminated. 
        ## If we will not use yield for loop will start from 0 again when function is get called
        yield X[start: start+batch_size, :] , y[start: start+batch_size]

In [26]:
## X_train.shape[1] will give the no. of pixels(784)
## X_train.shape[0] will give the no of image
img_placeholder = tf.placeholder(dtype=tf.float32, shape=(None, X_train.shape[1]))

## Here label has only one of the value. so shape=(None, 1)
labels = tf.placeholder(dtype=tf.int32, shape=(None, 1))

## Define a placeholder for the learning rate
learning_rate_ph = tf.placeholder(dtype=tf.float32)

In [28]:
## Reshape the data set because tensorflow takes 4 Dimensional data. 
## 1 for batch size
## 2 for Height
## 3 for Width
## 4 for color channel(RGB)

## For our this dataset we have only 2 D
## 1 for batch size
## 2 for flatten image pixels (here our image of 1-D)

## So we need to resize our image to 4-D

def nn():
    
    input_2d = tf.reshape(img_placeholder,[-1,28,28,1], name="Reshape") ## will change to 28x28 matrices, -1 will replace with batch size
    
    ## 1- stride, 8-no. of output, 5-height weight of kernel. Output is 24x24x8
    l1_1 = tf.layers.conv2d(input_2d,8,5,1,activation=tf.nn.relu, name="First_Conv_layer")
    
    ## Reduce the size of output by half 23x23x8
    l1_2 = tf.layers.average_pooling2d(l1_1,2,1, name="average_pooling_layer")
    
    ## size of image will be 21x21x8
    l1_3 = tf.layers.conv2d(l1_2,8,3,1,activation=tf.nn.relu, name="Second_Conv_layer")
    
    ## size of image will be 10x10x8
    l1_4 = tf.layers.max_pooling2d(l1_3,2,2, name="max_pooling_layer")
    
    ## total of 800 pixels
    l1_5 = tf.layers.flatten(l1_4, name="flatten_layer")
    
    ## 
    l2   = tf.layers.dense(l1_5, 32, activation=tf.nn.relu, name="first_dense")
    
    ## output is 10 label
    out  = tf.layers.dense(l2,10, name="second_dense")
    
    # Define the cost
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=out, labels=tf.one_hot(labels,depth=10)))
    
    # Apply an optimizer
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate_ph).minimize(cost)
    
    return cost, optimizer, out

cost,optimizer,out = nn();

print("Cost: ",cost)
print("Optimizer: ",optimizer)
print("Out: ",out)

ValueError: Variable First_Conv_layer/kernel already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "C:\Users\prince.yadav\AppData\Local\Continuum\miniconda3\envs\semantic\lib\site-packages\tensorflow\python\framework\ops.py", line 1718, in __init__
    self._traceback = self._graph._extract_stack()  # pylint: disable=protected-access
  File "C:\Users\prince.yadav\AppData\Local\Continuum\miniconda3\envs\semantic\lib\site-packages\tensorflow\python\framework\ops.py", line 3392, in create_op
    op_def=op_def)
  File "C:\Users\prince.yadav\AppData\Local\Continuum\miniconda3\envs\semantic\lib\site-packages\tensorflow\python\framework\op_def_library.py", line 787, in _apply_op_helper
    op_def=op_def)


## Convolutional Neural Network